<a href="https://colab.research.google.com/github/hardik1408/CS771/blob/hardik/d3_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: open train_text_seq.csv

import pandas as pd
df_train = pd.read_csv('/content/train_text_seq.csv')
df_valid = pd.read_csv('/content/valid_text_seq.csv')
df_train.head()



,input_str,label
0,0000154364642718159661428002624223132284159626...,0
1,0004641596369515436422262614110471596262476161...,0
2,0001543626215965999614422464135806142624051159...,0
3,0000154364224641238614262159689561596284351061...,1
4,0004641899422154362069015966142624761262159661...,1


# 100%

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the datasets
df_train = pd.read_csv('/content/train_text_seq.csv')
df_valid = pd.read_csv('/content/valid_text_seq.csv')

# Extract input sequences and labels
X_train = df_train['input_str'].values
y_train = df_train['label'].values
X_valid = df_valid['input_str'].values
y_valid = df_valid['label'].values

# Encode the digits using LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(list('0123456789'))

# Convert input_str into sequences of encoded digits, removing the first three zeroes
def encode_sequence(sequence):
    return label_encoder.transform(list(sequence[3:]))  # Remove first 3 zeros

X_train_encoded = [encode_sequence(seq) for seq in X_train]
X_valid_encoded = [encode_sequence(seq) for seq in X_valid]

# Pad the sequences to ensure they all have the same length (47 in this case)
X_train_padded = pad_sequences(X_train_encoded, maxlen=47, padding='post')
X_valid_padded = pad_sequences(X_valid_encoded, maxlen=47, padding='post')

# Convert the labels to categorical (0 or 1)
y_train_categorical = to_categorical(y_train, num_classes=2)
y_valid_categorical = to_categorical(y_valid, num_classes=2)

# Build the LSTM model with more parameters while staying under 10,000
model = Sequential()
model.add(Embedding(input_dim=10, output_dim=16, input_length=47))  # 10 unique digits, embedding size 16
model.add(LSTM(32, return_sequences=True))  # Bidirectional LSTM with 32 units, return sequences
model.add(Dropout(0.3))  # Dropout layer to prevent overfitting
model.add(LSTM(16))  # Additional LSTM layer with 16 units
model.add(Dense(16, activation='relu'))  # Dense layer with 16 units
model.add(Dense(2, activation='softmax'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train_categorical,
          validation_data=(X_valid_padded, y_valid_categorical),
          epochs=150, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_valid_padded, y_valid_categorical)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')
model.summary()

Epoch 1/150


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


222/222 ━━━━━━━━━━━━━━━━━━━━ 13s 37ms/step - accuracy: 0.5034 - loss: 0.6934 - val_accuracy: 0.6012 - val_loss: 0.6846
Epoch 2/150
222/222 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - accuracy: 0.5874 - loss: 0.6736 - val_accuracy: 0.6401 - val_loss: 0.6340
Epoch 3/150
222/222 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - accuracy: 0.6160 - loss: 0.6491 - val_accuracy: 0.6196 - val_loss: 0.6398
Epoch 4/150
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - accuracy: 0.6298 - loss: 0.6422 - val_accuracy: 0.6360 - val_loss: 0.6218
Epoch 5/150
222/222 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - accuracy: 0.6433 - loss: 0.6343 - val_accuracy: 0.6503 - val_loss: 0.6210
Epoch 6/150
222/222 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - accuracy: 0.6362 - loss: 0.6336 - val_accuracy: 0.6585 - val_loss: 0.6256
Epoch 7/150
222/222 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.6322 - loss: 0.6328 - val_accuracy: 0.6789 - val_loss: 0.6051
Epoch 8/150
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - accuracy: 0.6549 - loss: 0.6211 - val_accu

# 80%

# 60%

# 40%

# 20%